In [ ]:
# Conecta con Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Git clone y configuración de DeOldify
!git clone https://github.com/jantic/DeOldify.git DeOldify

Cloning into 'DeOldify'...
remote: Enumerating objects: 2612, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (186/186), done.
remote: Total 2612 (delta 89), reused 208 (delta 73), pack-reused 2346
Receiving objects: 100% (2612/2612), 69.71 MiB | 28.89 MiB/s, done.
Resolving deltas: 100% (1172/1172), done.


In [ ]:
cd DeOldify

/content/DeOldify


In [ ]:
from deoldify import device
from deoldify.device_id import DeviceId

In [ ]:
# Configuración del dispositivo
device.set(device=DeviceId.GPU0)

<DeviceId.GPU0: 0>

In [ ]:
import torch
if not torch.cuda.is_available():
    print('GPU no disponible.')

GPU no disponible.


In [ ]:
pip install -r requirements-colab.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.3/237.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.4/631.4 kB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.1/354.1 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 14.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using c

In [ ]:
import fastai
from deoldify.visualize import *
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


NumExpr defaulting to 2 threads.


In [ ]:
!mkdir 'models'
!wget https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth -O ./models/ColorizeArtistic_gen.pth

mkdir: cannot create directory ‘models’: File exists
--2024-06-03 00:57:44--  https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth
Resolving data.deepai.org (data.deepai.org)... 185.93.1.249, 2400:52e0:1a00::894:1
Connecting to data.deepai.org (data.deepai.org)|185.93.1.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 255144681 (243M) [application/octet-stream]
Saving to: ‘./models/ColorizeArtistic_gen.pth’

./models/ColorizeAr 100%[===================>] 243.32M  5.13MB/s    in 50s     

2024-06-03 00:58:34 (4.91 MB/s) - ‘./models/ColorizeArtistic_gen.pth’ saved [255144681/255144681]



In [ ]:
colorizer = get_image_colorizer(artistic=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can als

In [ ]:
# Definir carpetas de entrada y salida
input_folder = '/content/drive/MyDrive/Cosas del semestre 7/PDI/Proyecto PDI/Imagenes/Procesado 1'
output_folder = '/content/drive/MyDrive/Cosas del semestre 7/PDI/Proyecto PDI/Imagenes/Procesado 2'

In [ ]:
# Asegurarse de que la carpeta de salida exista
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Función para convertir a escala de grises si es necesario
def ensure_grayscale(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Si la imagen original no está en escala de grises, la convertimos y guardamos temporalmente
    if len(image.shape) == 3 and image.shape[2] == 3:
        temp_gray_path = image_path.replace('.jpg', '_gray.jpg')
        cv2.imwrite(temp_gray_path, gray)
        return temp_gray_path
    return image_path

In [ ]:
# Procesar todas las imágenes en la carpeta de entrada
for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        input_path = os.path.join(input_folder, filename)
        gray_image_path = ensure_grayscale(input_path)

        # Colorear la imagen
        result = colorizer.get_transformed_image(path=gray_image_path, render_factor=35, watermarked=True)

        # Guardar la imagen coloreada en la carpeta de salida
        output_path = os.path.join(output_folder, filename)
        result.save(output_path)
        print(f'Procesado y guardado: {input_path} -> {output_path}')

print('Procesamiento completo.')

Procesado y guardado: /content/drive/MyDrive/Cosas del semestre 7/PDI/Proyecto PDI/Imagenes/Procesado 1/Foto 1.jpg -> /content/drive/MyDrive/Cosas del semestre 7/PDI/Proyecto PDI/Imagenes/Procesado 2/Foto 1.jpg
Procesado y guardado: /content/drive/MyDrive/Cosas del semestre 7/PDI/Proyecto PDI/Imagenes/Procesado 1/Foto 2.jpg -> /content/drive/MyDrive/Cosas del semestre 7/PDI/Proyecto PDI/Imagenes/Procesado 2/Foto 2.jpg
Procesado y guardado: /content/drive/MyDrive/Cosas del semestre 7/PDI/Proyecto PDI/Imagenes/Procesado 1/Foto 3.jpg -> /content/drive/MyDrive/Cosas del semestre 7/PDI/Proyecto PDI/Imagenes/Procesado 2/Foto 3.jpg
Procesado y guardado: /content/drive/MyDrive/Cosas del semestre 7/PDI/Proyecto PDI/Imagenes/Procesado 1/Foto 4.jpg -> /content/drive/MyDrive/Cosas del semestre 7/PDI/Proyecto PDI/Imagenes/Procesado 2/Foto 4.jpg
Procesado y guardado: /content/drive/MyDrive/Cosas del semestre 7/PDI/Proyecto PDI/Imagenes/Procesado 1/Foto 5.jpg -> /content/drive/MyDrive/Cosas del semes